In [10]:
!pip install torch torchvision
!pip install git+https://github.com/facebookresearch/detectron2.git

Defaulting to user installation because normal site-packages is not writeable
^C


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/facebookresearch/detectron2.git to c:\users\alaka\appdata\local\temp\pip-req-build-i_k98tw9
  Resolved https://github.com/facebookresearch/detectron2.git to commit 8d85329aed8506ea3672e3e208971345973ea761
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for detectron2: filename=detectron2-0.6-cp312-cp312-win_amd64.whl size=1201259 sha256=2e61c6e36e990e46c517ef9102e052a546d10b92a972a4e06b9ec5daa2654c64
  Stored in directory: C:\Users\alaka\AppData\Local\Temp\pip-ephem-wheel-cache-jraztfx9\wheels\d3\6e\bd\1969578f1456a6be2d6f083da65c669f450b23b8f3d1ac14c1
Successfully built detectron2


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git 'C:\Users\alaka\AppData\Local\Temp\pip-req-build-i_k98tw9'


In [11]:
!git clone https://github.com/facebookresearch/meshrcnn.git
%cd meshrcnn
!pip install -e .


e:\hacktwofuture\hack2future\meshrcnn\meshrcnn


Cloning into 'meshrcnn'...
C:\Users\alaka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///E:/hacktwofuture/hack2future/meshrcnn/meshrcnn
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of meshrcnn to determine which version is compatible with other requirements. This could take a while.


ERROR: Could not find a version that satisfies the requirement detectron2 (from meshrcnn) (from versions: none)
ERROR: No matching distribution found for detectron2


In [12]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname("hack2future/meshrcnn/meshrcnn_whole.ipynb"), '..'))
# sys.path.append(parent_dir)


from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Load the image
image_path = "/content/hi.jpg"
image = cv2.imread(image_path)

# Configure Mask R-CNN
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

# Run Mask R-CNN prediction
outputs = predictor(image)

# Visualize Mask R-CNN Results
v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(out.get_image()[:, :, ::-1])
plt.axis("off")
plt.show()

# Extract Mask R-CNN results for Mesh R-CNN
instances = outputs["instances"]
masks = instances.pred_masks.cpu().numpy()
boxes = instances.pred_boxes.tensor.cpu().numpy()
classes = instances.pred_classes.cpu().numpy()

# Initialize Mesh R-CNN
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/meshrcnn_R50_FPN.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/meshrcnn_R50_FPN.yaml")
mesh_predictor = DefaultPredictor(cfg)

# Iterate over each detected object to generate 3D meshes
for i, (mask, box, cls) in enumerate(zip(masks, boxes, classes)):
    print(f"Processing object {i+1} (Class: {cls})")
    
    # Mask and bounding box preprocessing for Mesh R-CNN
    # Create a masked image to pass to Mesh R-CNN
    masked_image = image.copy()
    masked_image[~mask] = 0  

    # Run Mesh R-CNN prediction on the masked image
    mesh_output = mesh_predictor(masked_image)
    
    v = Visualizer(masked_image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(mesh_output["instances"].to("cpu"))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.axis("off")
    plt.show()

print("Completed Mask R-CNN and Mesh R-CNN processing.")

model_final_f10217.pkl: 178MB [00:58, 3.02MB/s]                               


AttributeError: 'NoneType' object has no attribute 'shape'